<a href="https://colab.research.google.com/github/Ramaseshanr/anlp/blob/master/LSI_Physics_Problems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import os, gensim
from gensim.models import LsiModel
from gensim import models
import numpy as np
from gensim import corpora
from gensim.utils import lemmatize
from pprint import  pprint
import nltk
from nltk.corpus import stopwords
from urllib.request import urlopen
from gensim.utils import lemmatize

nltk.download('wordnet')
nltk.download('stopwords')



stop_words = stopwords.words('english') + ['km','m','seconds','h','hour','meter','meters','mi','mile',
                                           'miles','minute','minutes','ms','s²','________','second','sec','seconds','e','r','s',
                                           'hour','hours','cm','hr','b','min','n','nd','degree','degrees','v']

model_dir = '/home/'
physics_corpus_dir = 'https://raw.githubusercontent.com/Ramaseshanr/anlp/master/corpus/phy_corpus.txt'

import pandas as pd
cor = pd.read_csv(physics_corpus_dir, sep = '\n',header = None)[0]



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [37]:
import os, gensim
from gensim.models import LsiModel
from gensim import models
import numpy as np
from gensim import corpora
from gensim.utils import lemmatize
from pprint import  pprint
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from pprint import pprint
from gensim.parsing.preprocessing import remove_stopwords, stem_text

nltk.download('wordnet')

from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma


stop_words = stopwords.words('english') + ['km','m','seconds','h','hour','meter','meters','mi','mile',
                                           'miles','minute','minutes','ms','s²','________','second','sec','seconds','e','r','s',
                                           'hour','hours','cm','hr','b','min','n','nd','degree','degrees','v']
porter = PorterStemmer()

model_dir = '/home/'
physics_corpus_dir = 'https://raw.githubusercontent.com/Ramaseshanr/anlp/master/corpus/phy_corpus.txt'

import pandas as pd
cor = pd.read_csv(physics_corpus_dir, sep = '\n',header = None)[0]
        
def iter_documents():
  """Iterate over all documents, yielding a document (=list of utf8 tokens) at a time."""
  for document in cor:
    #doc = stem_text(document)
    yield gensim.utils.tokenize(document, lower=True)


def remove_stopwords(corpus):
    stop_ids = [
        corpus.dictionary.token2id[stopword]
        for stopword in stop_words
            if stopword in corpus.dictionary.token2id
        ]
    corpus.dictionary.filter_tokens(stop_ids)
    return corpus    
  


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [38]:
class build_my_corpus(object):
    def __init__(self):
        self.dictionary = gensim.corpora.Dictionary(iter_documents())
        self.dictionary.filter_extremes(no_below=1, keep_n=30000) # check API docs for pruning params

    def __iter__(self):
        for tokens in iter_documents():
            yield self.dictionary.doc2bow(tokens)

def build_corpus():
    corpus = build_my_corpus() # create a dictionary
    return corpus


def build_LSI_Model(corpus):
    model = LsiModel(corpus, id2word=corpus.dictionary,num_topics=10,decay=0.5)
    model.save(model_dir+'physics.model.gensim')

corpus = build_corpus()
lsi_model = build_LSI_Model(corpus)
phy_lsi_model = LsiModel.load(model_dir+'physics.model.gensim')
pprint(phy_lsi_model.print_topics())


[(0,
  '0.307*"it" + 0.296*"at" + 0.278*"velocity" + 0.243*"how" + 0.239*"in" + '
  '0.236*"what" + 0.229*"and" + 0.227*"from" + 0.179*"ground" + 0.172*"an"'),
 (1,
  '0.418*"it" + 0.300*"how" + -0.267*"car" + 0.254*"ground" + 0.228*"ball" + '
  '-0.204*"what" + -0.185*"at" + -0.170*"km" + -0.169*"velocity" + '
  '-0.162*"for"'),
 (2,
  '0.415*"velocity" + 0.315*"ball" + -0.258*"how" + -0.248*"in" + -0.239*"car" '
  '+ 0.209*"an" + 0.201*"initial" + 0.198*"with" + 0.167*"ground" + '
  '-0.165*"km"'),
 (3,
  '0.421*"car" + -0.400*"in" + -0.316*"and" + 0.281*"at" + 0.231*"ball" + '
  '-0.213*"you" + -0.174*"average" + 0.129*"from" + -0.126*"air" + '
  '-0.119*"an"'),
 (4,
  '0.495*"speed" + -0.306*"seconds" + -0.279*"velocity" + 0.253*"ball" + '
  '-0.243*"car" + -0.198*"in" + 0.187*"an" + 0.185*"with" + 0.136*"take" + '
  '0.121*"time"'),
 (5,
  '0.400*"ball" + 0.307*"what" + 0.286*"at" + 0.284*"you" + 0.263*"in" + '
  '-0.226*"velocity" + -0.174*"and" + 0.152*"are" + -0.150*"find" + '


In [0]:
texts = [
  [word for word in document.lower().split() if word not in stop_words]
    for document in cor]

In [40]:
c = [corpus.dictionary.doc2bow(text) for text in texts]
tfidf = models.TfidfModel(c)
print(tfidf)
corpus_tfidf = tfidf[c]

TfidfModel(num_docs=271, num_nnz=2759)


In [0]:
lsi = models.LsiModel(corpus_tfidf, id2word=corpus.dictionary, num_topics=2)  # initialize an LSI transformation
corpus_lsi = lsi[corpus_tfidf]

In [44]:
from gensim import similarities
doc = "car starts from rest"
vec_bow = corpus.dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]  # convert the query to LSI space
index = similarities.MatrixSimilarity(lsi[c])
sims = index[vec_lsi] 
sims = sorted(enumerate(sims), key=lambda item: -item[1])
for index, similarity in sims:
  print(similarity,cor[index])

0.99996233 A car can accelerate from 0 to 60 mph in 15 seconds. What is its acceleration?
0.99966323 A car accelerates uniformly from rest at a rate of +10 m/s2. How far will the car travel in 4 seconds? 
0.99912953 How far will a car travel in 25 min at 12 m/s? 
0.99766636 A race car accelerates uniformly from 18.5 m/s to 46.1 m/s in 2.47 seconds. Determine the acceleration of the car. 
0.9975078 A car accelerates from 0 km/h to 100 km/h in 5 seconds. It can decelerate at 5 m/s, if the driver applies the brakes fully. What is the acceleration of the car?
0.99680257 A race car accelerates uniformly from 18.5 m/s to 46.1 m/s in 2.47 seconds. Determine the acceleration of the car and the distance traveled.
0.99614894 A car starts from rest and accelerates uniformly for 5.21 seconds over a distance of 110 m.  Determine the acceleration of the car.
0.995688 How far (in meters) will you travel in 3 minutes running at a rate of 6 m/s?
0.99557966 A car travels between 2 towns 60 miles apart i

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
